In [ ]:
import pandas as pd
from libpytunes import Library
import mutagen
import os
import sqlalchemy
import uuid
from datetime import datetime
import shutil

In [ ]:
ITUNES_LIBRARY_LOCATION = r"C:/Users/rdoge/Music/iTunes/iTunes Music Library.xml"
OLD_NAVIDROME_LIBRARY_LOCATION = "navidrome.db"
NEW_NAVIDROME_LIBRARY_LOCATION = "navidrome_new.db"

In [ ]:
itunes = Library(ITUNES_LIBRARY_LOCATION)

In [1]:
# for song_id, song in itunes.songs.items():
#     print(f"{song_id}: {song.artist} - {song.name}")

In [6]:
song = itunes.songs[21500]

In [7]:
song.name

'True Faith'

In [8]:
song.rating

In [9]:
for prop, value in vars(song).items():
    print(f"{prop}: {value}")

name: True Faith
work: None
movement_number: None
movement_count: None
movement_name: None
track_id: 21500
artist: New Order
album_artist: New Order
composer: None
album: Substance 1987
genre: New Wave
kind: AAC audio file
size: 8698980
total_time: 355093
track_number: 12
track_count: 12
disc_number: 1
disc_count: 2
year: 1987
date_modified: time.struct_time(tm_year=2022, tm_mon=2, tm_mday=12, tm_hour=6, tm_min=53, tm_sec=0, tm_wday=5, tm_yday=43, tm_isdst=-1)
date_added: time.struct_time(tm_year=2022, tm_mon=2, tm_mday=12, tm_hour=6, tm_min=53, tm_sec=28, tm_wday=5, tm_yday=43, tm_isdst=-1)
bit_rate: 193
sample_rate: 44100
comments: None
rating: None
rating_computed: False
play_count: None
album_rating: None
album_rating_computed: False
persistent_id: 9D61999A0F23D49A
location_escaped: file://localhost/F:/music/iTunes%20AAC%20Transcodes/Songs/New%20Order%20-%20True%20Faith.m4a
location: F:/music/iTunes AAC Transcodes/Songs/New Order - True Faith.m4a
compilation: False
lastplayed: None

In [10]:
f"{song.artist} - {song.name} | {song.album} ({song.year}) {song.total_time // (1000*60)}:{(song.total_time//1000)%60:02} {'♥ ' if song.loved else ''}{('☆' if song.rating_computed else '★') * (song.rating // 20 if song.rating else 0)}"

'New Order - True Faith | Substance 1987 (1987) 5:55 '

In [11]:
song_file_metadata = mutagen.File(song.location)

In [12]:
# song_file_metadata["TXXX:MusicBrainz Release Track Id"][0]
# bytes(song_file_metadata["----:com.apple.iTunes:MusicBrainz Track Id"][0]).decode("utf-8")

In [13]:
itunes_songs = {}
counter = 0
for song_id, song in itunes.songs.items():
    counter += 1
    if counter % 1000 == 0:
        print(f"Song {counter}/{len(itunes.songs)}")
    if any([song.podcast, song.protected, song.movie, song.playlist_only, song.apple_music]):
        continue
    musicbrainz_track_id = None
    in_itunes_music = None
    if song.location:
        if os.path.exists(song.location):
            song_file_metadata = mutagen.File(song.location)
            if song_file_metadata.get("----:com.apple.iTunes:MusicBrainz Track Id"):
                    musicbrainz_track_id = bytes(song_file_metadata["----:com.apple.iTunes:MusicBrainz Track Id"][0]).decode("utf-8")
            elif song_file_metadata.get("TXXX:MusicBrainz Release Track Id"):
                    musicbrainz_track_id = song_file_metadata["TXXX:MusicBrainz Release Track Id"][0]
        in_itunes_music = song.location.startswith("C:/Users/rdoge/Music/iTunes/iTunes Music/")
    lastplayed = None
    if song.lastplayed:
        lastplayed = datetime(*song.lastplayed[:6])
    itunes_songs[song_id] = {"name": song.name, "artist": song.artist, "rating_itunes": song.rating, "rating_computed": song.rating_computed, "play_count": song.play_count, "location": song.location, "in_itunes_music": in_itunes_music, "musicbrainz_track_id": musicbrainz_track_id, "play_count": song.play_count, "lastplayed": lastplayed}

Song 1000/7919
Song 2000/7919
Song 3000/7919
Song 4000/7919
Song 5000/7919
Song 6000/7919
Song 7000/7919


In [14]:
itunes_df = pd.DataFrame.from_dict(itunes_songs, orient='index')

In [15]:
itunes_df["rating_in_stars"] = pd.NA

In [16]:
itunes_df["rating_in_stars"] = itunes_df[(itunes_df["rating_itunes"].notnull()) & (~itunes_df["rating_computed"])]["rating_itunes"] // 20

In [17]:
itunes_df

,name,artist,rating_itunes,rating_computed,play_count,location,in_itunes_music,musicbrainz_track_id,lastplayed,rating_in_stars
5752,Come Together,The Beatles,60.0,False,21.0,C:/Users/rdoge/Music/iTunes/iTunes Music/The B...,True,None,2012-05-03 03:37:51,3.0
5754,Something,The Beatles,60.0,False,32.0,C:/Users/rdoge/Music/iTunes/iTunes Music/The B...,True,None,2022-02-25 17:30:45,3.0
5756,Maxwell's Silver Hammer,The Beatles,60.0,True,25.0,C:/Users/rdoge/Music/iTunes/iTunes Music/The B...,True,None,2013-06-26 18:12:12,NaN
5758,Oh! Darling,The Beatles,60.0,True,23.0,C:/Users/rdoge/Music/iTunes/iTunes Music/The B...,True,None,2012-05-31 15:18:08,NaN
5760,Octopus's Garden,The Beatles,60.0,False,16.0,C:/Users/rdoge/Music/iTunes/iTunes Music/The B...,True,None,2015-01-09 05:28:48,3.0
...,...,...,...,...,...,...,...,...,...,...
21562,Shizam,"Zed Bias, Stylo G & Scrufizzer",60.0,False,NaN,F:/music/songs/Anything remotely electronic/Ze...,False,c79707e6-ab22-4921-95d2-cf299e8ae009,NaT,3.0
21564,Under,Pleasure P,NaN,False,NaN,F:/music/songs/R&B - Newer Soul/Pleasure P - U...,False,28347420-2696-37ee-80e6-b32b61fd9998,NaT,NaN
21566,Same Shit Different Day,Havoc,NaN,False,1.0,F:/music/songs/Anything remotely electronic/Ha...,False,fb3704db-4384-453e-9c86-bf1795e9e0ea,2022-03-06 14:41:41,NaN
21570,Sofa (Nonchalant),Chapeaumelon,80.0,False,3.0,F:/music/songs/Rock/Chapeaumelon - Sofa (Nonch...,False,e3025af6-7ffb-4b46-8e7f-00c210d2bbf9,2022-03-06 03:21:58,4.0


In [18]:
itunes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7292 entries, 5752 to 21600
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   name                  7292 non-null   object        
 1   artist                7260 non-null   object        
 2   rating_itunes         3676 non-null   float64       
 3   rating_computed       7292 non-null   bool          
 4   play_count            5427 non-null   float64       
 5   location              7198 non-null   object        
 6   in_itunes_music       7197 non-null   object        
 7   musicbrainz_track_id  4891 non-null   object        
 8   lastplayed            5417 non-null   datetime64[ns]
 9   rating_in_stars       1571 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(3), object(5)
memory usage: 576.8+ KB


In [19]:
# itunes_df.to_parquet("itunes_songs.parquet")

In [20]:
shutil.copy2(OLD_NAVIDROME_LIBRARY_LOCATION, NEW_NAVIDROME_LIBRARY_LOCATION)

'navidrome_new.db'

In [21]:
# Create your connection.
engine = sqlalchemy.create_engine("sqlite:///" + OLD_NAVIDROME_LIBRARY_LOCATION)

with engine.connect() as conn:
    navidrome_song_df = pd.read_sql_query("SELECT * FROM media_file WHERE mbz_track_id IS NOT NULL", conn)
    annotation_df = pd.read_sql_table("annotation", conn)

engine.dispose()

In [22]:
navidrome_song_df

,id,path,title,album,artist,artist_id,album_artist,album_id,has_cover_art,track_number,...,mbz_artist_id,mbz_album_artist_id,mbz_album_type,mbz_album_comment,catalog_num,comment,lyrics,bpm,channels,order_title
0,37862d0706fd9a47f239b247549afc7a,/music/albums/2562 - Aerial/009-2562-kameleon.mp3,Kameleon,Aerial,2562,dbab2adc8f9d078009ee3fa810bea142,2562,826dbbca8298a8d82024cc465d50a9b4,1,9,...,5a557973-2e65-4429-a60d-a9886bfbac6a,5a557973-2e65-4429-a60d-a9886bfbac6a,,,TECCD004,,,0,2,Kameleon
1,e55643332f0db7b2bd909f077feb8b7e,/music/albums/2562 - Aerial/001-2562-redux.mp3,Redux,Aerial,2562,dbab2adc8f9d078009ee3fa810bea142,2562,826dbbca8298a8d82024cc465d50a9b4,1,1,...,5a557973-2e65-4429-a60d-a9886bfbac6a,5a557973-2e65-4429-a60d-a9886bfbac6a,,,TECCD004,,,0,2,Redux
2,3c68cda5f18522b66f3d011bd59d4871,/music/albums/2562 - Aerial/002-2562-morvern.mp3,Morvern,Aerial,2562,dbab2adc8f9d078009ee3fa810bea142,2562,826dbbca8298a8d82024cc465d50a9b4,1,2,...,5a557973-2e65-4429-a60d-a9886bfbac6a,5a557973-2e65-4429-a60d-a9886bfbac6a,,,TECCD004,,,0,2,Morvern
3,4a72a9da42608586bf431afc526593e8,/music/albums/2562 - Aerial/004-2562-channel_t...,Channel Two,Aerial,2562,dbab2adc8f9d078009ee3fa810bea142,2562,826dbbca8298a8d82024cc465d50a9b4,1,4,...,5a557973-2e65-4429-a60d-a9886bfbac6a,5a557973-2e65-4429-a60d-a9886bfbac6a,,,TECCD004,,,0,2,Channel Two
4,50050f743195440153cf293df40e5d06,/music/albums/2562 - Aerial/003-2562-moog_dub.mp3,Moog Dub,Aerial,2562,dbab2adc8f9d078009ee3fa810bea142,2562,826dbbca8298a8d82024cc465d50a9b4,1,3,...,5a557973-2e65-4429-a60d-a9886bfbac6a,5a557973-2e65-4429-a60d-a9886bfbac6a,,,TECCD004,,,0,2,Moog Dub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7618,1c4af6f080857ef66bcdfed87a9affc3,/music/songs/Rock/Chapeaumelon - Sofa (Nonchal...,Sofa (Nonchalant),Chapeaumelon,Chapeaumelon,07c2a76a8dda4cfb3c7864ab4bc54943,Chapeaumelon,e3051b1d3be26922228aabb20ac764e4,1,2,...,5d98f67f-3562-42ce-a779-f8b5d0ba75b1,5d98f67f-3562-42ce-a779-f8b5d0ba75b1,,,,,,0,2,Sofa (Nonchalant)
7619,11e3d6deb715308ce6e4253731475bca,/music/songs/Rock/Gregorian - Ave Satani (The ...,Ave Satani (The Omen),The Dark Side,Gregorian,a56c75f6ac705510397b519cbfe3d39d,Gregorian,e677758f69eeacc8510af7a63ff783e3,1,12,...,18a0d72d-0be0-4196-a340-035163bfae1c,18a0d72d-0be0-4196-a340-035163bfae1c,,,0158082ERE,,,0,2,Ave Satani (The Omen)
7620,09ff19f4d93403596288affe538d1bb4,/music/songs/Anything remotely electronic/Eric...,Call On Me (radio edit),Call On Me,Eric Prydz,570934c95f1d141104627e6e47f94ebc,Eric Prydz,e8c1e6680e0012c225a21c75d00b8305,1,1,...,35dac7d2-0b1f-470f-9a5a-c53c8821f6d6,35dac7d2-0b1f-470f-9a5a-c53c8821f6d6,,,DATA68CDS,,"Call on me, call on me\nCall on me, call on me...",0,2,Call On Me (radio edit)
7621,293d01200d2b2b775b0568f21acf591b,/music/songs/Anything remotely electronic/Wago...,Musipal,Musipal,Wagon Christ,57123dde9844ac8d811d998426dce99b,Wagon Christ,9c8e04a0e6efdc32539e5114f0958ad9,0,2,...,,,,,,,,0,2,Musipal


In [23]:
navidrome_song_df.columns

Index(['id', 'path', 'title', 'album', 'artist', 'artist_id', 'album_artist',
       'album_id', 'has_cover_art', 'track_number', 'disc_number', 'year',
       'size', 'suffix', 'duration', 'bit_rate', 'genre', 'compilation',
       'created_at', 'updated_at', 'full_text', 'album_artist_id',
       'order_album_name', 'order_album_artist_name', 'order_artist_name',
       'sort_album_name', 'sort_artist_name', 'sort_album_artist_name',
       'sort_title', 'disc_subtitle', 'mbz_track_id', 'mbz_album_id',
       'mbz_artist_id', 'mbz_album_artist_id', 'mbz_album_type',
       'mbz_album_comment', 'catalog_num', 'comment', 'lyrics', 'bpm',
       'channels', 'order_title'],
      dtype='object')

In [24]:
annotation_df

,ann_id,user_id,item_id,item_type,play_count,play_date,rating,starred,starred_at
0,c4234b4b-ef97-4719-b9a3-b045ba287a76,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,f12cb91a7885f1e027e0e0a65a52ddbf,media_file,1,2021-11-30 01:59:43,3,False,NaT
1,01442847-631d-410d-b063-19c7de57f4c9,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,01754fd08306328a5c786d986d56f2d1,album,2,2021-11-30 21:29:44,0,False,NaT
2,6eaabbf8-6b01-4380-8d13-ff3d1c4578ec,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,83cba0493179ae3d5198cfadbbebf603,media_file,1,2021-11-30 02:11:51,3,True,2021-11-30 02:11:59
3,a0a363f1-d752-4cf0-8ce7-fe570b7c4579,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b40558e18a88b22be1709b473f46771f,media_file,0,NaT,3,False,NaT
4,5dc610c3-5f30-4cf9-89d9-eeea80fc28a2,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,d1358cfce5e18b1adef106f37428153e,media_file,0,NaT,3,False,NaT
...,...,...,...,...,...,...,...,...,...
287,56304a6f-8340-47b2-93c7-6186df0ebe72,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,e20cc302bad8a36bca21aeab6d15c052,album,1,2022-01-12 15:34:10,0,False,NaT
288,ecb7c430-145b-4961-b951-591d81442d4c,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,282804bf069cacf4844e7e853b3c9e14,artist,1,2022-01-12 15:34:10,0,False,NaT
289,3dbccf76-17b9-4827-b15c-5eb9c7eb5f62,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,11e8d7685077eaf7ecaf65daca09ae25,media_file,1,2022-01-12 15:39:54,0,False,NaT
290,915656cb-37e2-42f1-87a0-65573c126a63,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,7019c3f09e780f9e91e21246c4afc753,album,1,2022-01-12 15:39:54,0,False,NaT


In [25]:
merged_song_df = navidrome_song_df.merge(itunes_df, how="left", left_on="mbz_track_id", right_on="musicbrainz_track_id", suffixes=('_navidrome', '_itunes'))

In [26]:
rating_to_song = merged_song_df[merged_song_df["rating_in_stars"].notnull()][["id", "rating_in_stars", "play_count", "lastplayed"]]
rating_to_song["rating_in_stars"] = rating_to_song["rating_in_stars"].astype(int)
rating_to_song["play_count"] = rating_to_song["play_count"].fillna(0).astype(int)
rating_to_song = rating_to_song.rename(columns={"id": "item_id", "rating_in_stars": "rating", "lastplayed": "play_date"})

In [27]:
rating_to_song

,item_id,rating,play_count,play_date
32,b40558e18a88b22be1709b473f46771f,3,1,2021-11-16 21:56:20
129,ffc81d9d1b466032ebeda801c31be67d,4,0,NaT
130,7943b316060e57b174ca09cf76ee8ad3,3,1,2022-01-12 05:03:54
173,5e58db3d07a7b7405b70e2d221f04705,3,1,2020-11-19 22:21:09
176,35cea62323cd1d9cbf4f87a87cc44fde,4,6,2021-10-20 04:20:27
...,...,...,...,...
7594,040c7b744439c7b0db4b6cfe122ec43c,3,0,NaT
7605,e95ecaf4e5242401a41ba615313827e9,3,0,NaT
7609,806d963d48d0b6eb54d54c51da97e42d,3,1,2022-02-14 03:51:38
7613,b6eb95893e90cdec2328a923cb8838fa,3,1,2022-02-13 02:15:37


In [28]:
annotation_rows_to_add = rating_to_song
annotation_rows_to_add["user_id"] = annotation_df["user_id"].value_counts().index[0]
annotation_rows_to_add['ann_id'] = [uuid.uuid4() for _ in range(len(annotation_rows_to_add.index))]
annotation_rows_to_add["item_type"] = "media_file"
annotation_rows_to_add["starred"] = 0
annotation_rows_to_add["starred_at"] = None
annotation_rows_to_add["starred_at"] = pd.to_datetime(annotation_rows_to_add["starred_at"])

In [29]:
annotation_rows_to_add = annotation_rows_to_add[["ann_id", "user_id", "item_id", "item_type", "play_count", "play_date", "rating", "starred", "starred_at"]]

In [30]:
annotation_rows_to_add

,ann_id,user_id,item_id,item_type,play_count,play_date,rating,starred,starred_at
32,5edd480f-e95f-4373-8c96-999d6012a418,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b40558e18a88b22be1709b473f46771f,media_file,1,2021-11-16 21:56:20,3,0,NaT
129,e5af5996-b47b-4217-81fe-60e050c71ee9,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,ffc81d9d1b466032ebeda801c31be67d,media_file,0,NaT,4,0,NaT
130,7239b6f0-1ff6-4148-844e-ae74bcdba705,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,7943b316060e57b174ca09cf76ee8ad3,media_file,1,2022-01-12 05:03:54,3,0,NaT
173,6d3afe36-314f-49a9-b062-9a03c8bd7b96,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,5e58db3d07a7b7405b70e2d221f04705,media_file,1,2020-11-19 22:21:09,3,0,NaT
176,36184f75-3e99-4f43-8ee0-b8f434e5376f,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,35cea62323cd1d9cbf4f87a87cc44fde,media_file,6,2021-10-20 04:20:27,4,0,NaT
...,...,...,...,...,...,...,...,...,...
7594,d25daee6-e086-4729-ab84-58265e25411a,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,040c7b744439c7b0db4b6cfe122ec43c,media_file,0,NaT,3,0,NaT
7605,f9b1dcdc-e54c-46a4-b92c-070509fd3447,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,e95ecaf4e5242401a41ba615313827e9,media_file,0,NaT,3,0,NaT
7609,af2cd0a9-6849-41cb-9cc7-de3d925c5047,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,806d963d48d0b6eb54d54c51da97e42d,media_file,1,2022-02-14 03:51:38,3,0,NaT
7613,cc51444f-24f2-4293-b44e-a2e43db1624d,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b6eb95893e90cdec2328a923cb8838fa,media_file,1,2022-02-13 02:15:37,3,0,NaT


In [31]:
merged_annotation_df = annotation_df.merge(annotation_rows_to_add, how="outer", on="item_id", suffixes=("_nvd", "_itu"))

In [32]:
merged_annotation_df

,ann_id_nvd,user_id_nvd,item_id,item_type_nvd,play_count_nvd,play_date_nvd,rating_nvd,starred_nvd,starred_at_nvd,ann_id_itu,user_id_itu,item_type_itu,play_count_itu,play_date_itu,rating_itu,starred_itu,starred_at_itu
0,c4234b4b-ef97-4719-b9a3-b045ba287a76,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,f12cb91a7885f1e027e0e0a65a52ddbf,media_file,1.0,2021-11-30 01:59:43,3.0,False,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
1,01442847-631d-410d-b063-19c7de57f4c9,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,01754fd08306328a5c786d986d56f2d1,album,2.0,2021-11-30 21:29:44,0.0,False,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
2,6eaabbf8-6b01-4380-8d13-ff3d1c4578ec,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,83cba0493179ae3d5198cfadbbebf603,media_file,1.0,2021-11-30 02:11:51,3.0,True,2021-11-30 02:11:59,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
3,a0a363f1-d752-4cf0-8ce7-fe570b7c4579,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b40558e18a88b22be1709b473f46771f,media_file,0.0,NaT,3.0,False,NaT,5edd480f-e95f-4373-8c96-999d6012a418,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,media_file,1.0,2021-11-16 21:56:20,3.0,0.0,NaT
4,5dc610c3-5f30-4cf9-89d9-eeea80fc28a2,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,d1358cfce5e18b1adef106f37428153e,media_file,0.0,NaT,3.0,False,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,NaN,NaN,040c7b744439c7b0db4b6cfe122ec43c,NaN,NaN,NaT,NaN,NaN,NaT,d25daee6-e086-4729-ab84-58265e25411a,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,media_file,0.0,NaT,3.0,0.0,NaT
646,NaN,NaN,e95ecaf4e5242401a41ba615313827e9,NaN,NaN,NaT,NaN,NaN,NaT,f9b1dcdc-e54c-46a4-b92c-070509fd3447,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,media_file,0.0,NaT,3.0,0.0,NaT
647,NaN,NaN,806d963d48d0b6eb54d54c51da97e42d,NaN,NaN,NaT,NaN,NaN,NaT,af2cd0a9-6849-41cb-9cc7-de3d925c5047,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,media_file,1.0,2022-02-14 03:51:38,3.0,0.0,NaT
648,NaN,NaN,b6eb95893e90cdec2328a923cb8838fa,NaN,NaN,NaT,NaN,NaN,NaT,cc51444f-24f2-4293-b44e-a2e43db1624d,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,media_file,1.0,2022-02-13 02:15:37,3.0,0.0,NaT


In [33]:
merged_annotation_df["ann_id"] = merged_annotation_df["ann_id_nvd"].combine_first(merged_annotation_df["ann_id_itu"])
merged_annotation_df["user_id"] = merged_annotation_df["user_id_nvd"].combine_first(merged_annotation_df["user_id_itu"])
merged_annotation_df["item_type"] = merged_annotation_df["item_type_nvd"].combine_first(merged_annotation_df["item_type_itu"])
merged_annotation_df["play_count"] = merged_annotation_df["play_count_nvd"].combine_first(merged_annotation_df["play_count_itu"])
merged_annotation_df["play_date"] = merged_annotation_df["play_date_nvd"].combine_first(merged_annotation_df["play_date_itu"])
merged_annotation_df["rating"] = merged_annotation_df["rating_nvd"].combine_first(merged_annotation_df["rating_itu"])
merged_annotation_df["starred"] = merged_annotation_df["starred_nvd"].combine_first(merged_annotation_df["starred_itu"])
merged_annotation_df["starred_at"] = merged_annotation_df["starred_at_nvd"].combine_first(merged_annotation_df["starred_at_itu"])

In [34]:
select_criteria = (merged_annotation_df["play_count_nvd"] * 5) <= (merged_annotation_df["play_count_itu"])
merged_annotation_df.loc[select_criteria, "play_count"] = merged_annotation_df.loc[select_criteria, "play_count_nvd"] + merged_annotation_df.loc[select_criteria, "play_count_itu"]

In [35]:
select_criteria = merged_annotation_df["play_date_nvd"] != merged_annotation_df["play_date_itu"]
merged_annotation_df[select_criteria]["play_count"] += 1

In [36]:
merged_annotation_df = merged_annotation_df[["ann_id", "user_id", "item_id", "item_type", "play_count", "play_date", "rating", "starred", "starred_at"]]

In [37]:
merged_annotation_df = merged_annotation_df.copy()

In [38]:
merged_annotation_df

,ann_id,user_id,item_id,item_type,play_count,play_date,rating,starred,starred_at
0,c4234b4b-ef97-4719-b9a3-b045ba287a76,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,f12cb91a7885f1e027e0e0a65a52ddbf,media_file,1.0,2021-11-30 01:59:43,3.0,False,NaT
1,01442847-631d-410d-b063-19c7de57f4c9,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,01754fd08306328a5c786d986d56f2d1,album,2.0,2021-11-30 21:29:44,0.0,False,NaT
2,6eaabbf8-6b01-4380-8d13-ff3d1c4578ec,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,83cba0493179ae3d5198cfadbbebf603,media_file,1.0,2021-11-30 02:11:51,3.0,True,2021-11-30 02:11:59
3,a0a363f1-d752-4cf0-8ce7-fe570b7c4579,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b40558e18a88b22be1709b473f46771f,media_file,1.0,2021-11-16 21:56:20,3.0,False,NaT
4,5dc610c3-5f30-4cf9-89d9-eeea80fc28a2,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,d1358cfce5e18b1adef106f37428153e,media_file,0.0,NaT,3.0,False,NaT
...,...,...,...,...,...,...,...,...,...
645,d25daee6-e086-4729-ab84-58265e25411a,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,040c7b744439c7b0db4b6cfe122ec43c,media_file,0.0,NaT,3.0,0.0,NaT
646,f9b1dcdc-e54c-46a4-b92c-070509fd3447,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,e95ecaf4e5242401a41ba615313827e9,media_file,0.0,NaT,3.0,0.0,NaT
647,af2cd0a9-6849-41cb-9cc7-de3d925c5047,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,806d963d48d0b6eb54d54c51da97e42d,media_file,1.0,2022-02-14 03:51:38,3.0,0.0,NaT
648,cc51444f-24f2-4293-b44e-a2e43db1624d,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b6eb95893e90cdec2328a923cb8838fa,media_file,1.0,2022-02-13 02:15:37,3.0,0.0,NaT


In [39]:
merged_annotation_df

,ann_id,user_id,item_id,item_type,play_count,play_date,rating,starred,starred_at
0,c4234b4b-ef97-4719-b9a3-b045ba287a76,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,f12cb91a7885f1e027e0e0a65a52ddbf,media_file,1.0,2021-11-30 01:59:43,3.0,False,NaT
1,01442847-631d-410d-b063-19c7de57f4c9,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,01754fd08306328a5c786d986d56f2d1,album,2.0,2021-11-30 21:29:44,0.0,False,NaT
2,6eaabbf8-6b01-4380-8d13-ff3d1c4578ec,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,83cba0493179ae3d5198cfadbbebf603,media_file,1.0,2021-11-30 02:11:51,3.0,True,2021-11-30 02:11:59
3,a0a363f1-d752-4cf0-8ce7-fe570b7c4579,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b40558e18a88b22be1709b473f46771f,media_file,1.0,2021-11-16 21:56:20,3.0,False,NaT
4,5dc610c3-5f30-4cf9-89d9-eeea80fc28a2,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,d1358cfce5e18b1adef106f37428153e,media_file,0.0,NaT,3.0,False,NaT
...,...,...,...,...,...,...,...,...,...
645,d25daee6-e086-4729-ab84-58265e25411a,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,040c7b744439c7b0db4b6cfe122ec43c,media_file,0.0,NaT,3.0,0.0,NaT
646,f9b1dcdc-e54c-46a4-b92c-070509fd3447,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,e95ecaf4e5242401a41ba615313827e9,media_file,0.0,NaT,3.0,0.0,NaT
647,af2cd0a9-6849-41cb-9cc7-de3d925c5047,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,806d963d48d0b6eb54d54c51da97e42d,media_file,1.0,2022-02-14 03:51:38,3.0,0.0,NaT
648,cc51444f-24f2-4293-b44e-a2e43db1624d,2e7d8278-2533-45e2-a1b9-0a702f87ddbf,b6eb95893e90cdec2328a923cb8838fa,media_file,1.0,2022-02-13 02:15:37,3.0,0.0,NaT


In [40]:
merged_annotation_df = merged_annotation_df.astype(annotation_df.dtypes.to_dict())
merged_annotation_df = merged_annotation_df.astype({"ann_id": "string", "user_id": "string", "item_id": "string", "item_type": "string"})

In [41]:
annotation_df.dtypes

ann_id                object
user_id               object
item_id               object
item_type             object
play_count             int64
play_date     datetime64[ns]
rating                 int64
starred                 bool
starred_at    datetime64[ns]
dtype: object

In [42]:
merged_annotation_df.dtypes

ann_id                string
user_id               string
item_id               string
item_type             string
play_count             int64
play_date     datetime64[ns]
rating                 int64
starred                 bool
starred_at    datetime64[ns]
dtype: object

In [43]:
# Create your connection.
engine = sqlalchemy.create_engine("sqlite:///" + NEW_NAVIDROME_LIBRARY_LOCATION)

with engine.connect() as conn:
    merged_annotation_df.to_sql(name='annotation', con=conn, if_exists='replace', index=False)

engine.dispose()